In [1]:
import numpy as np
from scipy.integrate import simps
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

fn_1s5 = "1s5_old.txt"
fn_2p9 = "2p9_old.txt"
#pr=""
#while pr.find("a")!=-1 or pr.find("e")!=-1:

def lorentz(A, s, b, w):
    return np.array([A * b / (2 * np.pi * ((w[i] - s) ** 2 + (b / 2) ** 2)) for i in range(len(w))])

def maxwell(v, T):
    mu = 3.6364
    k = 8.314e7
    return np.array([4 * np.pi * (v[i] ** 2) * ((mu / (2 * np.pi * k * T)) ** 1.5) * np.exp(-mu * v[i] ** 2 / (2 * k * T)) for i in range(len(v))])

en1s5_file = open(fn_1s5, "r")
en2p9_file = open(fn_2p9, "r")
m1s5 = [line.strip() for line in en1s5_file.readlines()]
m2p9 = [line.strip() for line in en2p9_file.readlines()]
en1s5_file.close()
en2p9_file.close()
keys_1s5 = m1s5[0].split()
data_1s5 = {}
keys_2p9 = m2p9[0].split()
data_2p9 = {}
g = len(keys_1s5)
for j in range(len(keys_1s5)):
    temp_array = np.array([])
    for i in range(1,len(m1s5)):
        temp_str = m1s5[i].split()
        #print(temp_str)
        temp_array = np.append(temp_array,float(temp_str[j]))
    data_1s5[keys_1s5[j]]=temp_array
    #if j > 0: plt.plot(data_1s5[keys_1s5[0]],data_1s5[keys_1s5[j]])
for j in range(len(keys_2p9)):
    temp_array = np.array([])
    for i in range(1,len(m2p9)):
        temp_str = m2p9[i].split()
        temp_array = np.append(temp_array,float(temp_str[j]))
    data_2p9[keys_2p9[j]]=temp_array
    #if j > 0: plt.plot(data_2p9[keys_2p9[0]],data_2p9[keys_2p9[j]])
dm_file = open("Intens.txt", "r")
strdm = [line.strip() for line in dm_file.readlines()]
strdm = [s.split() for s in strdm]
rule = {}
for j in range(1, len(keys_1s5)):
    subrule = {}
    for k in range(1, len(keys_2p9)):
        subrule[keys_2p9[k]] = float(strdm[j][k])
    rule[keys_1s5[j]] = subrule
trans = {}
weights = {}
for kup in keys_2p9[1:]:
    for kdown in keys_1s5[1:]:
        if rule[kdown][kup] != 0:
            k = kup + "_" + kdown
            trans[k] = data_2p9[kup] - data_1s5[kdown]
            weights[k] = rule[kdown][kup]
trans['R'] = data_2p9[keys_2p9[0]] * 1e-8
# print(trans['R'])
dzetas = {}
dzetab = {}
T = np.arange(300,800,50)
v = np.arange(1e3, 2e8, 1e3)
c = 2.998E10
z = np.arange(-20,20,0.05) * 1e-8
rho = np.arange(2,20,0.05) * 1e-8
dzetas['T'] = T
dzetab['T'] = T
outsname = "shift_test.txt"
outbname = "broad_test.txt"
open(outsname, "w").close()
dzetasf = open(outsname, "w")
open(outbname, "w").close()
dzetabf = open(outbname, "w")
r = trans['R']
for t in trans.keys():
    sigb = np.zeros(len(v))
    sigs = np.zeros(len(v))
    if t!='R':
        w = trans[t]
        dw = (w - w[len(w) - 1]*np.ones(len(w))) * 2 * np.pi * c
        sp = CubicSpline(r, dw)
        # plt.plot(r[:len(r)-10], dw[:len(r)-10])
        # plt.plot(r[:len(r)-10], sp(r[:len(r)-10]), 'bo')
        # plt.show()
        # print(w[-1])
        # dw = (-1E-42 / (r ** 6)) + (1E-42 * np.ones(len(r)) / (r[0] ** 6))
        # print(dw)
        # print()
        eta = np.zeros(len(r))
        for i in range(len(r)):
            # считаем интеграл эта, который определяет сдвиг по фазе при пролете
            deta = sp(np.sqrt(r[i] ** 2 + z ** 2))
            # print(deta)
            eta[i] = simps(deta, z)
        # print(eta)
        sp = CubicSpline(r, eta)
        for j in range(len(v)):
            # print(1 - np.cos(2 * eta / v[j]))
            # print(eta / v[j])
            # plt.plot(rho, (1 - np.cos(sp(rho) / v[j])) * 2 * np.pi * rho)
            # plt.plot(rho, np.sin(sp(rho) / v[j]) * 2 * np.pi * rho)
            # plt.show()
            sigb[j] = simps((1 - np.cos(sp(rho) / v[j])) * 2 * np.pi * rho, rho)
            sigs[j] = simps(np.sin(sp(rho) / v[j]) * 2 * np.pi * rho, rho)
        # print(sigb)
        t1 = np.zeros(len(T))
        t2 = np.zeros(len(T))
        for k in range(len(T)):
            dist = maxwell(v, T[k])
            # plt.plot(np.log10(v), sigb * v * dist)
            # plt.show()
            t1[k] = simps(sigb * v * dist, v)
            t2[k] = simps(sigs * v * dist, v)
        dzetab[t] = t1
        dzetas[t] = t2
for k in dzetas.keys():
    dzetasf.write(k+"\t")
dzetasf.write("\n")
for j in range(len(T)):
    for k in dzetas.keys():
        if k == "T":
            dzetasf.write(str(dzetas[k][j])+"\t")
        else:
            dzetasf.write("{0:.4}\t".format(dzetas[k][j]))
    dzetasf.write("\n")
dzetasf.close()
for k in dzetab.keys():
    dzetabf.write(k+"\t")
dzetabf.write("\n")
for j in range(len(T)):
    for k in dzetab.keys():
        if k == "T":
            dzetabf.write(str(dzetab[k][j])+"\t")
        else:
            dzetabf.write("{0:.4}\t".format(dzetab[k][j]))
    dzetabf.write("\n")
dzetabf.close()
w = np.arange(-2e11, 2e11, 5e7)
results1 = []
results2 = []
results3 = []
results4 = []
results5 = []
for j in range(len(T)):
    spectrum1 = np.zeros(len(w))
    spectrum2 = np.zeros(len(w))
    spectrum3 = np.zeros(len(w))
    spectrum4 = np.zeros(len(w))
    spectrum5 = np.zeros(len(w))
    for k in dzetas.keys():
        if k != "T":
            spectrum1 += lorentz(weights[k], dzetas[k][j] * 2e18, dzetab[k][j] * 2e18, w)
            spectrum2 += lorentz(weights[k], dzetas[k][j] * 4e18, dzetab[k][j] * 4e18, w)
            spectrum3 += lorentz(weights[k], dzetas[k][j] * 6e18, dzetab[k][j] * 6e18, w)
            spectrum4 += lorentz(weights[k], dzetas[k][j] * 8e18, dzetab[k][j] * 8e18, w)
            spectrum5 += lorentz(weights[k], dzetas[k][j] * 1e19, dzetab[k][j] * 1e19, w)
    results1.append(spectrum1)
    results2.append(spectrum2)
    results3.append(spectrum3)
    results4.append(spectrum4)
    results5.append(spectrum5)
open("spectrum1_test.txt", "w").close()
spf1 = open("spectrum1_test.txt", "w")
open("spectrum2_test.txt", "w").close()
spf2 = open("spectrum2_test.txt", "w")
open("spectrum3_test.txt", "w").close()
spf3 = open("spectrum3_test.txt", "w")
open("spectrum4_test.txt", "w").close()
spf4 = open("spectrum4_test.txt", "w")
open("spectrum5_test.txt", "w").close()
spf5 = open("spectrum5_test.txt", "w")
spf1.write("w")
spf2.write("w")
spf3.write("w")
spf4.write("w")
spf5.write("w")
for j in range(len(T)):
    spf1.write("\t" + str(T[j]))
    spf2.write("\t" + str(T[j]))
    spf3.write("\t" + str(T[j]))
    spf4.write("\t" + str(T[j]))
    spf5.write("\t" + str(T[j]))
spf1.write("\n")
spf2.write("\n")
spf3.write("\n")
spf4.write("\n")
spf5.write("\n")
for k in range(len(w)):
    spf1.write(str(w[k]))
    spf2.write(str(w[k]))
    spf3.write(str(w[k]))
    spf4.write(str(w[k]))
    spf5.write(str(w[k]))
    for j in range(len(T)):
        spf1.write("\t" + str(results1[j][k]))
        spf2.write("\t" + str(results2[j][k]))
        spf3.write("\t" + str(results3[j][k]))
        spf4.write("\t" + str(results4[j][k]))
        spf5.write("\t" + str(results5[j][k]))
    spf1.write("\n")
    spf2.write("\n")
    spf3.write("\n")
    spf4.write("\n")
    spf5.write("\n")
spf1.close()
spf2.close()
spf3.close()
spf4.close()
spf5.close()

KeyboardInterrupt: 